# Basic Q&A System

### Sentence Retrival

#### Preprocessing 

In [9]:
import math,numpy,json,re,nltk,csv,time,re,os.path,sys,ast,itertools
# from gensim import corpora
from operator import add
from nltk.tokenize import word_tokenize,RegexpTokenizer
from nltk.tag import StanfordNERTagger
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from nltk import FreqDist, DictionaryProbDist

#from sklearn import metrics
#from math import sqrt
#from collections import OrderedDict
#from nltk.stem.wordnet import WordNetLemmatizer as WNL
#from sklearn.feature_extraction import DictVectorizer

import math,numpy,json,re,nltk
import time,re,os.path,sys
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from collections import defaultdict
from numpy import multiply
from math import sqrt
from nltk.tokenize import word_tokenize
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
import os.path as path
from collections import OrderedDict
from nltk.stem.wordnet import WordNetLemmatizer as WNL
from sklearn.feature_extraction import DictVectorizer
from nltk import FreqDist, DictionaryProbDist

from nltk.tag.stanford import StanfordPOSTagger
from nltk.stem import PorterStemmer

from nltk.parse.stanford import StanfordDependencyParser
import string
path_to_jar = 'data/stanford-parser.jar'
path_to_models_jar = 'data/stanford-parser-3.7.0-models.jar'
st=PorterStemmer()


#### Load data from json file

In [10]:
t0 = time.time()
dataset = []
train_path = os.path.abspath('data/QA_dev.json')

dataset = []
with open(train_path) as f:
    for line in f:
        dataset+=(json.loads(line))
print "Import Successful "
print "There are totally", len(dataset),'documents in this dataset'

Import Successful 
There are totally 40 documents in this dataset


#### Build tf-idf model 

In [11]:
stopwords = set(nltk.corpus.stopwords.words('english')) # wrap in a set() (see below)
stemmer = nltk.stem.PorterStemmer() 

def my_tokenizer(doc):
    terms = set()
    tokenizer = RegexpTokenizer(r'\w+(?:[-./]\w+)?')
    for token in tokenizer.tokenize(doc):
        if token not in stopwords: # 'in' and 'not in' operations are much faster over sets that lists
            terms.add(stemmer.stem(token.lower()))
    return list(terms)

In [12]:
class tf_idf_Model:
    def __init__(self, collection):
        self.vectorizer = TfidfVectorizer(max_df=0.95, min_df=1, use_idf=True,norm='l2',stop_words=None, tokenizer=my_tokenizer)
        self.collection_matrix = self.vectorizer.fit_transform(document_collections)
        feature_array = self.vectorizer.get_feature_names()
        self.features = dict()
        for index in range(len(feature_array)):
            term = feature_array[index]
            self.features[term] = index

    def predict(self, queies,limit=3):
        predictions = [self.inverted_index_score(i,limit) for i in  queies]
        return predictions

    def inverted_index_score(self, query_sent,limit=1):
        """
        now we implement inverted index to handle query
        
        :param query_sent: 
        :return: 
        
        """
        query_words = my_tokenizer(query_sent)
        score = defaultdict(float)

        for w in query_words:
            try:
                col_i = self.features[w]
                inverted_ix = self.collection_matrix[:, col_i]
                for di in range(inverted_ix.shape[0]):
                    score[di] += inverted_ix[di, 0]
            except KeyError:
                pass

        index_score = sorted(score.items(), key=lambda (k, v): v, reverse=True)

        if index_score:
            doc_indexs = [i[0] for i in index_score[:limit]]
            return doc_indexs
        else:
            return []

#### Build BM25 model

In [40]:
class BM25_Model(object): 
    def __init__(self, document_collection, k1=1.5, b=0.75, k3=1.0, EPS=0.25, tokenizer=my_tokenizer): 
        self.tokenizer = tokenizer 
        self.document_collection_length = len(document_collection) 
        self.avg_doc_length = sum(map(lambda x: float(len(x)), document_collection)) / self.document_collection_length 
        self.document_collection = [self.tokenizer(doc) for doc in document_collection] 
        self.tf = [] 
        self.df = defaultdict(int) 
        self.bm25_idf = defaultdict(float) 
        self.average_idf = -1 
        self.k1 = k1 
        self.k3 = k3 
        self.EPSILON = EPS 
        self.b = b 
        self.inverted_index = defaultdict(list) 
        self.initialize() 
    def initialize(self):
        for index, document in enumerate(self.document_collection): 
            doc_term_freq = FreqDist(document) 
            self.tf.append(doc_term_freq) 
            
            for word, freq in doc_term_freq.items(): 
                self.df[word] += 1 
                self.inverted_index[word].append(index) 
        for word, freq in self.df.items(): 
            self.bm25_idf[word] = math.log(self.document_collection_length - freq + 0.5) - math.log(freq + 0.5)  
            self.average_idf = sum(map(lambda k: float(self.bm25_idf[k]), self.bm25_idf.keys())) / len(self.bm25_idf.keys()) 
    
    def predict(self, queryX, limit=1): 
        q_prediction = [] 
        for query in queryX: 
            answers = self.bm25_get_most_relevant(query)[:limit] 
            if len(answers) == 0:
                q_prediction.append([]) 
            else:
                q_prediction.append([i[0] for i in answers]) 
        return q_prediction 

    def bm25_get_most_relevant(self, query): 
        query_tks = self.tokenizer(query) 
        scores = defaultdict(float) 
        for q_token in query_tks: 
            for doc_index in self.inverted_index[q_token]: 
                idf = self.bm25_idf[q_token] if self.bm25_idf[q_token] >= 0 else self.EPSILON * self.average_idf 
                top = self.tf[doc_index][q_token] * (self.k1 + 1) 
                below = self.tf[doc_index][q_token] + self.k1 * (1 - self.b + self.b * self.document_collection_length / self.avg_doc_length) 
                scores[doc_index] += idf * top / below 
        prels = scores.items() 
        sorted_socres = sorted(prels, key=lambda (k, v): v, reverse=True) 
        return sorted_socres 

#### Build Language Model 

In [14]:
class LM_Model:
    def __init__(self,documents,a = 0.5 ,tokenizer=my_tokenizer): 
        self.tokenizer = tokenizer  
        self.alpha = a
        self.document_collection = [self.tokenizer(doc) for doc in documents] 
        self.document_corpus = list(itertools.chain.from_iterable(self.document_collection))
        self.corpus_term_prob = {}
        self.corpus_term_freq = FreqDist(self.document_corpus)
        self.vocabulary = self.corpus_term_freq.keys()
        self.lmp = []
        self.initialize()
    def initialize(self): 
        document_freq = [FreqDist(doc) for doc in self.document_collection]
        length_corpus = len(self.document_corpus)
        for term,occurs in self.corpus_term_freq.items():
            self.corpus_term_prob[term] = float(occurs)/float(length_corpus)
        for sent_freq in document_freq:
            tempDict = {}
            for term in self.vocabulary:
                upper = sent_freq.get(term,0) + self.alpha*self.corpus_term_prob.get(term,0)
                below = self.corpus_term_freq.get(term,0) + self.alpha
                tempDict[term] = float(upper)/float(below)
            self.lmp.append(tempDict)
    def get_lm_socres(self,Query):
        doc_socres = []
        Query = my_tokenizer(Query)
        for doc_prob in  self.lmp:
            term_score = []
            for term in Query:
                if term in self.vocabulary:
                    term_score.append(doc_prob[term])
            query_score = numpy.product(term_score)
            doc_socres.append(query_score)
        sorted_score = sorted(list(enumerate(doc_socres)), key=lambda (k,v): v, reverse=True)
        doc_indexs = [i for i in sorted_score]
        return doc_indexs
    def predict(self,questions,limit = 3):
        predictions = [] 
        for query in questions: 
            answers = self.get_lm_socres(query)[:limit] 
            predictions.append([i[0] for i in answers]) 
        return predictions 

#### Output Various Model Results

In [21]:
def write_csv(csv_name,dataset,limit=1):
    csv_file = open(csv_name, mode='w',)
    fieldnames = ['document_ID', 'question_ID','question','prediction_ID','prediction_sentence']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames, )
    writer.writeheader()

    ddi = 0
    for document in dataset:
        evaluate_row = []
        questions = [i['question'] for i in document['qa']]
        model = document['model']
        predictions = model.predict(questions,limit)
        quest_index = 0
        for pred_index in predictions:
            drow = dict()
            drow['question_ID'] = quest_index
            drow['prediction_ID'] = pred_index
            evaluate_row.append(drow)
            quest_index += 1
        doc_sents = document['sentences']
        for r in evaluate_row:
            r['document_ID'] = ddi
            r['question'] = questions[r['question_ID']].encode('utf-8')
            if len(r['prediction_ID']) != 0:
                r['prediction_sentence'] = doc_sents[r['prediction_ID'][0]].encode('utf-8')
            else:
                print 'error prediction',ddi,r['question_ID'],r['question']
            writer.writerow(r)
        ddi += 1

In [42]:
#build model for each document collaction
# for document in dataset:
#     document_collections = document['sentences']
#     document['model'] = tf_idf_Model(document_collections)
# write_csv('data/tf_idf_dev_results.csv',dataset,1)

for document in dataset:
    document_collections_sents = document['sentences']
    document['model'] = BM25_Model(document_collections_sents)
write_csv('data/bm25_dev_predictions.csv',dataset,1)

# for document in dataset:
#     document_collections_sents = document['sentences']
#     document['model'] = LM_Model(document_collections_sents)
# write_csv('data/lm_dev_predictions.csv',dataset,1)

error prediction 11 783 Where did it open?
error prediction 18 182 What did the actof of milno do?
error prediction 25 245 When was Chanakya alive?


#### Import different result and compare predictions from CSV file

In [45]:
def give_prediction_values(filename,devdata = False):
    preds = []
    with open(filename) as csvfile:
        readCSV = csv.DictReader(csvfile, delimiter=',')
        for row in readCSV:
            tempDict = {}
            tempDict['DocID'] = int(row['document_ID'])
            tempDict['Predictions'] = ast.literal_eval(row['prediction_ID'])
            tempDict['QuestionIndex'] = int(row['question_ID'])
            tempDict['Question'] = row['question']
            preds.append(tempDict)
    count = 0
    bingo = 0
    if devdata:
        for d in preds:
            guess = d['Predictions'] 
            doc_i = d['DocID']
            qus_i = d['QuestionIndex']
            act_i = dataset[doc_i]['qa'][qus_i]['answer_sentence']
            if act_i in guess:
                bingo += 1
            count += 1
        print "correctness results :",float(bingo)/float(count)
    return preds

In [46]:
p = give_prediction_values('data/bm25_dev_predictions.csv',devdata=True)

correctness results : 0.651305683564


### Entity Extraction

In [ ]:
def json_load_byteified(file_handle):
    return _byteify(json.load(file_handle, object_hook=_byteify),ignore_dicts=True)

def _byteify(data, ignore_dicts = False):
    # if this is a unicode string, return its string representation
    if isinstance(data, unicode):
        return data.encode('utf-8')
    # if this is a list of values, return list of byteified values
    if isinstance(data, list):
        return [ _byteify(item, ignore_dicts=True) for item in data ]
    # if this is a dictionary, return dictionary of byteified keys and values
    # but only if we haven't already byteified it
    if isinstance(data, dict) and not ignore_dicts:
        return {
            _byteify(key, ignore_dicts=True): _byteify(value, ignore_dicts=True)
            for key, value in data.iteritems()
        }
    # if it's anything else, return it in its original form
    return data

with open("data/QA_test.json") as json_file:
    json_data = json_load_byteified(json_file)
print 'import success'

In [ ]:
cwd = os.getcwd()
st = StanfordNERTagger(cwd+'\data\english.all.3class.distsim.crf.ser.gz',cwd+'\data\stanford-ner.jar')

if not os.path.isfile("NERtest.json"):    
    start = time.time()
    progressT = len(json_data)    
    listOfDocument=[]
    i=0
    for jd in json_data:
        aList=[]        
        aList.append(st.tag_sents([word_tokenize(re.sub(',', '',re.sub('[^a-zA-Z0-9-_*., ]', ' ',x['question']))) for x in jd['qa']]))
        #remove the below file if running on test set
        #aList.extend([st.tag_sents([word_tokenize(re.sub(',', '',re.sub('[^a-zA-Z0-9-_*., ]', ' ',x['answer']))) for x in jd['qa']])])
        aList.append(st.tag_sents([word_tokenize(re.sub(',', '',re.sub('[^a-zA-Z0-9-_*., ]', ' ',x))) for x in jd['sentences']]))
        listOfDocument.append(aList)
        i+=1
        sys.stdout.write('\r')
        sys.stdout.write("%d%%" % (i*100/progressT))
        sys.stdout.flush()    
    for document in range(0,len(listOfDocument)):
        #change [2] to [1] if test set
        for sentence in range(0,len(listOfDocument[document][1])):
            for word in range(0,len(listOfDocument[document][1][sentence])):   
                listOfDocument[document][1][sentence][word]= (listOfDocument[document][1][sentence][word][0],listOfDocument[document][1][sentence][word][1] if not listOfDocument[document][1][sentence][word][0].isdigit() else u'NUMBER')
    with open('NERtest.json', 'w') as outfile:
        json.dump(listOfDocument, outfile)
    end = time.time()
    print '\nTime spending:',end - start    
else:    
    print 'there is a file'
with open("NERtest.json") as json_file:
        json_dataNER = json_load_byteified(json_file)

### Design answer rank system

In [22]:
class answer_rank():
    def __init__(self,NER,POS,QA):
        self.NER = NER
        self.POS = POS
        self.QA = QA
        self.json_data = None
        self.json_dataOrg = None
        self.json_dataPOS = None
        self.json_dataPOSOrg = None
        self.dictDoc = {}
        self.dictDocOrg = {}
        self.initialize()

        
    def initialize(self):
        with open(self.NER) as json_file:
            self.json_data = self.json_load_byteified(json_file)    
        with open(self.POS) as json_file:
            self.json_dataPOS = self.json_load_byteified(json_file)
        with open(self.QA) as json_file:
            self.json_dataOrg = self.json_load_byteified(json_file)
        with open(self.POS) as json_file:
            self.json_dataPOSOrg = self.json_load_byteified(json_file)
        for document in range(len(self.json_data)):
            for thing in range(len(self.json_data[document])):
                for sentence in range(len(self.json_data[document][thing])):
                    for word in range(len(self.json_data[document][thing][sentence])):
                        self.json_data[document][thing][sentence][word][1] = 'U'\
                        if word!=0 and self.json_data[document][thing][sentence][word][0][0].isupper()\
                        and self.json_data[document][thing][sentence][word][1]=='O'\
                        else self.json_data[document][thing][sentence][word][1]
                        
                        self.json_data[document][thing][sentence][word][0] = self.json_data[document][thing][sentence][word][0].lower()
        
                        if self.json_data[document][thing][sentence][word][0] \
                        in ['one','two','three','four','five','six','seven','eight','nine','ten','zero']\
                        or self.isfloat(self.json_data[document][thing][sentence][word][0]):
                            self.json_data[document][thing][sentence][word][1] = 'NUMBER'
        for document in range(len(self.json_dataPOS)):
            for thing in range(len(self.json_dataPOS[document])):
                for sentence in range(len(self.json_dataPOS[document][thing])):
                    for word in range(len(self.json_dataPOS[document][thing][sentence])):
                        self.json_dataPOS[document][thing][sentence][word][0] = self.json_dataPOS[document][thing][sentence][word][0].lower()
                        if self.have_number(self.json_dataPOS[document][thing][sentence][word][0]):
                            self.json_dataPOS[document][thing][sentence][word][1] = 'CD'
        print 'NER json file import successful'
        
        
    def json_load_byteified(self,file_handle):
        return self._byteify(json.load(file_handle, object_hook=self._byteify),ignore_dicts=True)

    def _byteify(self,data, ignore_dicts = False):
        # if this is a unicode string, return its string representation
        if isinstance(data, unicode):
            return data.encode('utf-8')
        # if this is a list of values, return list of byteified values
        if isinstance(data, list):
            return [self._byteify(item, ignore_dicts=True) for item in data ]
        # if this is a dictionary, return dictionary of byteified keys and values
        # but only if we haven't already byteified it
        if isinstance(data, dict) and not ignore_dicts:
            return {self._byteify(key, ignore_dicts=True): self._byteify(value, ignore_dicts=True)
                    for key, value in data.iteritems()}
        # if it's anything else, return it in its original form
        return data
    
    def have_number(self,s):
        return any(i.isdigit() for i in s)

    def isfloat(self,value):
        try:
            float(value)
            return True
        except ValueError:
            return False

    def readCSV(self,filename,l):
        csv_file = filename
        i=0
        answerSecondFilter = []
        dictDocOrg={}
        #question={}
        with open(csv_file, 'rb') as csvfile:
            readCSV = csv.DictReader(csvfile, delimiter=',')
            for row in readCSV:
                document_i = int(row['document_ID'])
                question_i = int(row['question_ID'])
                filteredlistOfNERSentence = []
                question_type = []
                predictionList = ast.literal_eval(row['prediction_ID'])
                prediction_index = predictionList[0] if len(predictionList)!=0 else 1
                try:
                    question_type = self.detectQuestion(document_i,question_i,prediction_index,l) 

                except: 
                    print "Question type error for",document_i,question_i,prediction_index
                #question[document_i,question_i] = row['question']        
                self.dictDoc[document_i,question_i]=prediction_index,question_type                                              


    
    def detectQuestion(self,d_i, q_i, p_i, l):
        openclassword=[]
        kindOfAnswer = []
        questionPOS = self.json_dataPOS[d_i][0][q_i]
        questionNER = self.json_data[d_i][0][q_i]        
        answerPOS = self.json_dataPOS[d_i][l][p_i]
        answerNER = self.json_data[d_i][l][p_i]
        specialcommand=[]
        #print questionNER, answerPOS
        originalWithout = [x[0] for x in questionPOS]
        originalWithoutA = [x[0] for x in answerPOS]
        getIndexOfWH = [y for y,x in enumerate(questionPOS) if 'W' in x[1]]
        #if wh
        if len(getIndexOfWH) != 0:
            getPOS = ''.join([x[1][0] for x in questionPOS])        

            #not at last word
            if getIndexOfWH[0]+1!=len(questionPOS):            
                searchWordAfterWh1 = re.search('W(.*?)V', getPOS, re.IGNORECASE)
                if searchWordAfterWh1:
                    if len(searchWordAfterWh1.group(1))!=0:
                        openclassword.append([searchWordAfterWh1.start()+len(searchWordAfterWh1.group(1))])
                    else:
                        openclassword.append([])
                else:
                    searchWordAfterWh1 = re.search('W(.*?)', getPOS, re.IGNORECASE)
                    openclassword.append([searchWordAfterWh1.start()+len(searchWordAfterWh1.group(0))])            
                frontPart = range(0,searchWordAfterWh1.start())
                backPart = range(searchWordAfterWh1.start()+len(searchWordAfterWh1.group(0)),len(getPOS))                        
                openclassword.append(frontPart+backPart)      
                #print openclassword
            else:
                openclassword=[[],[y for y,x in enumerate(questionPOS)][:-1]]
            #remove Stopwords        
            openclassword[1] = [x for x in openclassword[1] if questionPOS[x][0] not in stopwords]
            if len(openclassword[0])!=0:            
                numberIndicator = ['year','length','percentage', 'many','much']            
                for x in numberIndicator: 
                    for y in originalWithout[getIndexOfWH[0]:openclassword[0][0]+2]:
                        if x in y:
                            kindOfAnswer = ['NUMBER'] if len(kindOfAnswer)==0 else kindOfAnswer
                            specialcommand.append(x)
                personIndicator = ['name']
                for x in personIndicator:
                    for y in originalWithout[getIndexOfWH[0]:openclassword[0][0]+2]:
                        if x in y:
                            kindOfAnswer = ['PERSON','ORGANIZATION'] if len(kindOfAnswer)==0 else kindOfAnswer
                placeIndicator = ['location','place','country','city','area']
                for x in placeIndicator:
                    for y in originalWithout[getIndexOfWH[0]:openclassword[0][0]+2]:
                        if x in y:
                            kindOfAnswer = ['LOCATION'] if len(kindOfAnswer)==0 else kindOfAnswer
                            specialcommand=['location']
                if len(kindOfAnswer)==0:    
                    kindOfAnswer = ['O','U']
            else:
                numberIndicator = ['when']            
                personIndicator = ['who','whom','whose','name']
                placeIndicator = ['where','located']
                if originalWithout[getIndexOfWH[0]] in numberIndicator:
                    kindOfAnswer = ['NUMBER']
                    specialcommand = ['year']
                elif originalWithout[getIndexOfWH[0]] in personIndicator:
                    kindOfAnswer = ['PERSON','ORGANIZATION']
                elif originalWithout[getIndexOfWH[0]] in placeIndicator:
                    kindOfAnswer = ['LOCATION']
                    specialcommand=['location']
                else:
                    if len([y for y in openclassword[1] if questionPOS[y][0] in personIndicator])!=0:
                        kindOfAnswer = ['PERSON','ORGANIZATION']
                    elif len([y for y in openclassword[1] if questionPOS[y][0] in placeIndicator])!=0:
                        kindOfAnswer = ['LOCATION']
                        specialcommand=['location']
                    else: 
                        kindOfAnswer = ['O','U']
        else:
            openclassword = [[],[x for x,y in enumerate(originalWithout) if y not in stop]]    
            kindOfAnswer = ['O','U']
        #determine whether it requires number entity
        newList1=[]
        newList2=[]
        for x in range(len(openclassword[0])):
            if originalWithout[openclassword[0][x]] in originalWithoutA:            
                newList1.extend([f for f,h in enumerate(originalWithoutA) if originalWithout[openclassword[0][x]] == h])
        for x in range(len(openclassword[1])):
            if originalWithout[openclassword[1][x]] in originalWithoutA:
                newList2.extend([f for f,h in enumerate(originalWithoutA) if originalWithout[openclassword[1][x]] == h])
        openclassword[0]=newList1
        openclassword[1]=newList2
        return openclassword, kindOfAnswer,specialcommand   

    
    def createNP(self,answerToReturn,answerPOS,specialcommand,q,j):
        newAnswer = answerPOS[answerToReturn[1]][0]
        if (newAnswer.isdigit() and 'year' not in specialcommand):
            newAnswer = "{:,}".format(int(answerPOS[answerToReturn[1]][0]))
        for i in range(answerToReturn[1],0,-1):
            if answerToReturn[1] != 0:
                if answerPOS[i][1] =='NNP':
                    if 'NNP' in answerPOS[i-1][1] or (('DT' in answerPOS[i-1][1] or 'IN' in answerPOS[i-1][1]) \
                                                      and answerPOS[i-1][0] !='at' and 'location' in specialcommand):
                        newAnswer = answerPOS[i-1][0]+ " " +newAnswer
                    else:
                        break
                elif answerPOS[i][1] =='NN':
                    if 'JJ' in answerPOS[i-1][1] or 'DT' in answerPOS[i-1][1] or 'CD' in answerPOS[i-1][1] or answerPOS[i-1][1] =='NN':
                        newAnswer = answerPOS[i-1][0]+ " " +newAnswer
                    else:
                        break
                elif answerPOS[i][1] =='JJ':
                    if answerPOS[i-1][1] =='JJ':
                        newAnswer = answerPOS[i-1][0]+ " " +newAnswer
                    else:
                        break
                elif answerPOS[i][1] =='NNS':
                    if 'JJ' in answerPOS[i-1][1]:
                        newAnswer = answerPOS[i-1][0]+ " " +newAnswer
                    else:
                        break
                elif 'DT' in answerPOS[i][1] and 'location' not in specialcommand:
                    if 'TO' in answerPOS[i-1][1]:
                        newAnswer = answerPOS[i-1][0]+ " " +newAnswer
                    else:
                        break
                else:
                    if answerPOS[i][1] !='CD' and answerPOS[i][1] !='RB':
                        if 'NN' in answerPOS[i-1][1] or 'JJ' in answerPOS[i-1][1] or 'RB' in answerPOS[i-1][1]:
                            newAnswer = answerPOS[i-1][0]+ " " +newAnswer
                        else:
                            break
                    else:
                        break
            else:
                break
        for i in range(answerToReturn[1],len(answerPOS)-1):
            if answerToReturn[1] != len(answerPOS)-1:
                if answerPOS[i][1] =='NNP':                
                    if 'NNP' in answerPOS[i+1][1] or 'CC' in answerPOS[i+1][1] \
                    or 'IN' in answerPOS[i+1][1] or 'TO' in answerPOS[i+1][1] \
                    or ('NN' in answerPOS[i+1][1] and 'location' in specialcommand):                                        
                        newAnswer = newAnswer+' '+answerPOS[i+1][0]
                    else:                    
                        break
                elif answerPOS[i][1] =='CC':
                    if 'NNP' in answerPOS[i+1][1] or 'NN' == answerPOS[i+1][1]:
                        newAnswer = newAnswer+' '+answerPOS[i+1][0]
                    else:                    
                        break
                elif answerPOS[i][1] =='CD':
                    if 'CD' in answerPOS[i+1][1]:                 
                        newAnswer = newAnswer+' '+answerPOS[i+1][0]
                    else:                    
                        break
                elif answerPOS[i][1] =='TO' or answerPOS[i][1] =='DT':
                    if 'NN' in answerPOS[i+1][1] or 'RB' in answerPOS[i+1][1]:
                        newAnswer = newAnswer+' '+answerPOS[i+1][0]
                    else:
                        break 
                elif answerPOS[i][1] =='IN' and 'location' not in specialcommand:
                    if answerPOS[i+1][1] =='DT' or answerPOS[i+1][1] =='NNP' or answerPOS[i+1][1] =='TO' or answerPOS[i+1][1] =='CD':
                        newAnswer = newAnswer+' '+answerPOS[i+1][0]
                    else:
                        break
                elif answerPOS[i][1] =='JJ':
                    if 'NNS' in answerPOS[i+1][1] or 'NN' == answerPOS[i+1][1] or answerPOS[i+1][1] =='JJ':
                        newAnswer = newAnswer+' '+answerPOS[i+1][0]
                    else:
                        break
                elif answerPOS[i][1] =='NN':
                    if 'NN' in answerPOS[i+1][1] or 'JJ' in answerPOS[i+1][1] or 'IN' in answerPOS[i+1][1] or 'CC' in answerPOS[i+1][1]:
                        newAnswer = newAnswer+' '+answerPOS[i+1][0]
                    else:
                        break
                elif answerPOS[i][1] =='NNS':
                    if 'IN' in answerPOS[i+1][1] and 'ORG' in specialcommand:
                        newAnswer = newAnswer+' '+answerPOS[i+1][0]
                    else:
                        break
                else:
                    break
            else:
                break

        if newAnswer.split()[-1]=='along':
            newAnswer=newAnswer[:-6]
        if newAnswer.split()[-1]=='and':
            newAnswer=newAnswer[:-4]
        if newAnswer.split()[-1]=='for':
            newAnswer=newAnswer[:-4]
        if newAnswer.split()[-1]=='but':
            newAnswer=newAnswer[:-4]
        if newAnswer.split()[-1]=='in':
            newAnswer=newAnswer[:-3]
        if newAnswer.split()[-1]=='as':
            newAnswer=newAnswer[:-3]
        if newAnswer.split()[-1]=='at':
            newAnswer=newAnswer[:-3]
        if newAnswer.split()[-1]=='on':
            newAnswer=newAnswer[:-3]
        if newAnswer.split()[-1]=='to':
            newAnswer=newAnswer[:-3]
        if newAnswer.split()[-1]=='while':
            newAnswer=newAnswer[:-6]
        if newAnswer.split()[-1]=='despite':
            newAnswer=newAnswer[:-8]
        if newAnswer.split()[-1]=='because':
            newAnswer=newAnswer[:-8]
        if newAnswer.split()[-1].isdigit() and len(newAnswer.split()[-1])==4 and 'year' in specialcommand:
            newAnswer=newAnswer.split()[-1]
        if newAnswer.split()[-1].isdigit() and 'location' in specialcommand:
            newAnswer=newAnswer.split()[-1]
        if len(specialcommand)!=0 and 'percentage' in specialcommand[0]:
            newAnswer+='%'
        if 'location' in specialcommand:
            bufferNewAnswer=[]        
            for x in newAnswer.split():
                for y in self.json_dataOrg[q]['sentences'][j].split():
                    if x in y:
                        bufferNewAnswer.append(y)
                        break
            newAnswer=' '.join(bufferNewAnswer)
            newAnswer = newAnswer[:-1] if not (newAnswer[-1].isalnum() or newAnswer[-1]=='%') else newAnswer
        newAnswer = newAnswer.replace(',','-COMMA-')
        return newAnswer
    
    def returnAnswer(self,i,j,l):
        answerListPOS =self.json_dataPOS[i][l][self.dictDoc[i,j][0]]    
        answerListNER = self.json_data[i][l][self.dictDoc[i,j][0]]    
        question_type = self.dictDoc[i,j][1][1]
        #print answerListPOS,question_type
        question_typeLocationInAnswer = [x for x,y in enumerate(answerListNER) \
                                         if ('CD' in answerListPOS[answerListNER.index(y)][1] \
                                             or 'NN' in answerListPOS[answerListNER.index(y)][1]) \
                                         and y[1] in question_type and y[0] not in stopwords \
                                         and x not in self.dictDoc[i,j][1][0][0] \
                                         and x not in self.dictDoc[i,j][1][0][1]]
        #print question_typeLocationInAnswer
        if len(question_typeLocationInAnswer) ==0:
            question_typeLocationInAnswer = [x for x,y in enumerate(answerListNER) \
                                             if ('CD' in answerListPOS[answerListNER.index(y)][1] \
                                                 or 'NN' in answerListPOS[answerListNER.index(y)][1]) \
                                             and y[0] not in stopwords and x not in self.dictDoc[i,j][1][0][0] \
                                             and x not in self.dictDoc[i,j][1][0][1]]
        #print question_typeLocationInAnswer,dictDoc[i,j][1][0][1], dictDoc[i,j][1][0][0]
        scoreList = 0
        if len(self.dictDoc[i,j][1][0][0])!=0:
            maxScore = sys.maxint        
            for x in question_typeLocationInAnswer:
                score = sum([math.fabs(z-x) for z in self.dictDoc[i,j][1][0][0]])
                if maxScore>score:
                    maxScore=score
                    scoreList = x
        else:
            maxScore = sys.maxint
            scoreList = 0
            for x in question_typeLocationInAnswer:
                score = sum([math.fabs(z-x) for z in self.dictDoc[i,j][1][0][1]])
                if maxScore>score:
                    maxScore=score
                    scoreList = x
        answerToReturn = (self.json_dataPOSOrg[i][l][self.dictDoc[i,j][0]][scoreList][0],scoreList)
        answerPOS = self.json_dataPOSOrg[i][l][self.dictDoc[i,j][0]]
        specialcommand = self.dictDoc[i,j][1][2]
        question_index = self.dictDoc[i,j][0]
        document_index = i
        answer = self.createNP(answerToReturn,answerPOS,specialcommand,document_index,question_index)
        return answer

    def writeToFile(self,filename,l):
        with open(filename, mode='wb',) as csv_file:
            if l == 2 :
                fieldnames = ['document_id','question_id','answer_predict',"answer_actual",'tag','sentence','predict','question_type','question']
            elif l == 1:
                fieldnames = ['id','answer']
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames,delimiter=',')
            writer.writeheader()
            k = 0
            doc_size=len(self.json_data)
            for i in range(0, doc_size):
                for j in range(0,len(self.json_data[i][0])):
                    k+=1            
                    dictToCSV={}
                    if l == 2:
                        dictToCSV['document_id'] = i
                        dictToCSV['question_id'] = j
                        try:
                            dictToCSV['answer_predict']= self.returnAnswer(i,j,l)
                        except:
                            print '\nErrors on dev set return answers',i,j
                        dictToCSV['answer_actual'] = self.json_dataOrg[i]['qa'][j]['answer']
                        dictToCSV['tag'] =self.dictDoc[i,j]
                        dictToCSV['question_type'] = self.dictDoc[i,j][1][1]
                        dictToCSV['question'] = self.json_dataOrg[i]['qa'][j]['question']
                        dictToCSV['sentence'] = self.json_dataOrg[i]['qa'][j]['answer_sentence']
                        dictToCSV['predict'] = self.dictDoc[i,j][0]
                    elif l == 1:
                        dictToCSV={}
                        dictToCSV['id'] = k
                        try:
                            dictToCSV['answer'] = self.returnAnswer(i,j,l)
                        except:
                            print '\nError on test return answers',i,j,l
                    writer.writerow(dictToCSV)    
                    csv_file.flush()
                sys.stdout.write('\r')
                sys.stdout.write(str(k))
                sys.stdout.flush()
        csv_file.close()
        print '\nsuccess'

In [21]:
qa = answer_rank("data/NERdev.json","data/POSdev.json","data/QA_dev.json")
qa.readCSV("data/bm25_dev_predictions.csv",2)
qa.writeToFile("data/bm25_dev_result.csv",2)

NER json file import successful
1194
Errors on dev set return answers 6 98
1482
Errors on dev set return answers 8 4
1796
Errors on dev set return answers 11 598
3456
Errors on dev set return answers 18 159

Errors on dev set return answers 18 180
5573
Errors on dev set return answers 29 242

Errors on dev set return answers 29 443
7266
Errors on dev set return answers 34 259
8463
succes


# Enhancement for Q&A systems

In [54]:
class BM25_S_Model(object): 
    def __init__(self, document_collection, k1=1.5, b=0.75, k3=1.0, EPS=0.25, tokenizer=my_tokenizer): 
        self.tokenizer = tokenizer 
        self.document_collection_length = len(document_collection) 
        self.avg_doc_length = sum(map(lambda x: float(len(x)), document_collection)) / self.document_collection_length 
        self.document_collection = [self.tokenizer(doc) for doc in document_collection] 
        self.tf = [] 
        self.df = defaultdict(int) 
        self.bm25_idf = defaultdict(float) 
        self.average_idf = -1 
        self.k1 = k1 
        self.k3 = k3 
        self.EPSILON = EPS 
        self.b = b 
        self.inverted_index = defaultdict(list) 
        self.initialize() 
    def initialize(self):
        for index, document in enumerate(self.document_collection): 
            doc_term_freq = FreqDist(document) 
            self.tf.append(doc_term_freq) 
            
            for word, freq in doc_term_freq.items(): 
                self.df[word] += 1 
                self.inverted_index[word].append(index) 
        for word, freq in self.df.items(): 
            self.bm25_idf[word] = math.log(self.document_collection_length - freq + 0.5) - math.log(freq + 0.5)  
            self.average_idf = sum(map(lambda k: float(self.bm25_idf[k]), self.bm25_idf.keys())) / len(self.bm25_idf.keys()) 
    
    def predict(self, queryX, limit=1): 
        q_prediction = [] 
        for query in queryX: 
            answers = self.bm25_get_most_relevant(query)[:limit] 
            if len(answers) == 0:
                q_prediction.append([]) 
            else:
                q_prediction.append([i[0] for i in answers]) 
        return q_prediction 

    def bm25_get_most_relevant(self, query): 
        query_tks = self.tokenizer(query) 
        scores = defaultdict(float) 
        for q_token in query_tks: 
            for doc_index in self.inverted_index[q_token]: 
                idf = self.bm25_idf[q_token] if self.bm25_idf[q_token] >= 0 else self.EPSILON * self.average_idf 
                top = self.tf[doc_index][q_token] * (self.k1 + 1) 
                below = self.tf[doc_index][q_token] + self.k1 * (1 - self.b + self.b * self.document_collection_length / self.avg_doc_length) 
                scores[doc_index] += idf * top / below 
        prels = scores.items() 
        sorted_socres = sorted(prels, key=lambda (k, v): v, reverse=True) 
        score1 = score2 = score3 = 0
        new = []
        if len(sorted_socres)  > 3:
            if  sorted_socres[0][0] != 0 and sorted_socres[1][0] != 0 and sorted_socres[2][0] != 0:
                left1,right1 =  - 1, sorted_socres[0][0] +1
                left2,right2 = sorted_socres[1][0] - 1, sorted_socres[1][0] +1
                left3,right3 = sorted_socres[2][0] - 1, sorted_socres[2][0] +1
                score1= 0.8*scores[left1]+0.2*scores[right1]
                score2 = 0.8*scores[left2]+0.2*scores[right2]
                score3 = 0.8*scores[left3]+0.2*scores[right3]
            new.append((sorted_socres[0][0],sorted_socres[0][1]+score1))
            new.append((sorted_socres[1][0],sorted_socres[1][1]+score2))
            new.append((sorted_socres[2][0],sorted_socres[2][1]+score3))
        if new :
            new_sort = sorted_socres = sorted(new, key=lambda (k, v): v, reverse=True) 
            return new_sort
        else:
            return sorted_socres 

In [55]:
for document in dataset:
    document_collections_sents = document['sentences']
    document['model'] = BM25_S_Model(document_collections_sents)
write_csv('data/bm25_dev_ss.csv',dataset,1)

error prediction 11 783 Where did it open?
error prediction 18 182 What did the actof of milno do?
error prediction 25 245 When was Chanakya alive?


In [56]:
u = give_prediction_values('data/bm25_dev_ss.csv',devdata = True)

correctness results : 0.574618929458
